In [ ]:
%load_ext fireducks.pandas
from openai import OpenAI
import pandas as pd
import numpy as np
from typing import List, Optional
import re
import json
from openai import OpenAI
import time
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
categor = pd.read_csv('category_tree.csv')
labeled_train = pd.read_parquet('unlabeled_train.parquet')
labeled_train.shape

In [ ]:
categor = pd.read_csv('category_tree.csv')

In [ ]:
def json_to_description(json_str):

    if not isinstance(json_str, str):
        return ""
    
    if not json_str.strip():
        return ""
        
    try:
        # Предварительная очистка строки
        json_str = json_str.strip()
        json_str = re.sub(r'"{2}', '"', json_str)
        json_str = re.sub(r'\\{5}\'', '"', json_str)  # \\\\\' -> "
        json_str = re.sub(r'\\{3}\'', '"', json_str)  # \\\' -> "
        json_str = re.sub(r'\\\'', '"', json_str)     # \' -> "
        json_str = re.sub(r'^"(.+)\'$', r'\1', json_str)

        attributes = json.loads(json_str)
        
            
        description: List[str] = []
        
        for item in attributes:
            if not isinstance(item, dict):
                continue
                
            name = item.get("attribute_name")
            value = item.get("attribute_value", "")
            
            if name:  # Добавляем только если есть название атрибута
                description.append(f"{name}: {value}")
        
        return " ".join(description)
        
    except json.JSONDecodeError as e:
        return ""
    except Exception as e:
        return ""


# Функция для объединения Title и Description
def combine_title_description(row):
    title = row["source_name"]
    description = row["description"]
    combined_text = f"{title} {description}"
    return combined_text

In [ ]:
labeled_train["description"] = labeled_train["attributes"].progress_apply(json_to_description)
labeled_train["combined_text"] = labeled_train.progress_apply(combine_title_description, axis=1)

In [ ]:
# Загрузка предыдущего прогресса, если есть
try:
    df_save = pd.read_csv('unLabel_Qwen2.5-7B.csv')
except FileNotFoundError:
    df_save = pd.DataFrame(columns=['hash_id', 'name', 'category_id'])
df_save.shape

In [ ]:
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="no-api-key-required"
)

start_categor = categor[categor['parent_id'].isna()]['cat_name'].tolist()
categor_dict = categor.groupby('parent_id')['cat_name'].apply(list).to_dict()
combined_texts = labeled_train['combined_text'].tolist()
source_name = labeled_train["source_name"].tolist()
hash_id = labeled_train["hash_id"].tolist()

cat_id_pred = []



start_index = df_save.shape[0]
count = 0

for text, name, hash_id in tqdm(
                                zip(combined_texts[start_index:], source_name[start_index:], hash_id[start_index:]), 
                                total=len(combined_texts[start_index:])):
    flag = True
    count += 1
    name = text
    current_parent = None  # Сбрасываем для каждого товара
    lst = "\n - ".join(start_categor)  # Начинаем с корневых категорий
    cat = ''
    while flag:
        try:
            response = client.chat.completions.create(
                model="Qwen/Qwen2.5-7B-Instruct",
                messages = [
                    {
                        "role": "system",
                        "content": (
                            "Ты полезный помощник по определению категории товара и списка категорий, который строго следует инструкции"
                        )
                    },
                    {
                        "role": "user",
                        "content": (
                            f"Товар '{text}'. "
                            f"Текущая категория: {cat}. "
                            f"Выбери наиболее подходящую подкатегорию из списка: {lst} для товара {text}. \n"
                            "Ответ должен быть строго только названием категории без лишних слов. "
                            f"Ты всегда должен выбрать из списка {lst} наиболее релевантную категорию."
                        )
                    }
                ],
                max_tokens=100,
                temperature=0.2
            )

            answer = response.choices[0].message.content.strip('.')
            cat = f'{cat}/{answer}' 
        except Exception as e:
            print(f"Ошибка: {e}")
            cat_id_pred.append(None)  # Сохраняем None при ошибке
            break

        try:
            # Поиск категории в текущем уровне
            if current_parent is not None:
                mask = (categor['cat_name'] == answer) & (categor['parent_id'] == current_parent)
            else:
                mask = (categor['cat_name'] == answer) & (categor['parent_id'].isna())
            
            category = categor.loc[mask]
            if category.empty:
                raise IndexError("Категория не найдена")
            
            current_parent = category['cat_id'].iloc[0]

            # Проверяем наличие подкатегорий
            subcategories = categor_dict.get(current_parent, [])
            if subcategories:
                lst = ",\n ".join(subcategories)
            else:
                flag = False
                cat_id_pred.append([hash_id, name, current_parent])
                
        except Exception as e:
            cat_id_pred.append([hash_id, name, current_parent if current_parent else np.nan])
            flag = False

    # Сохранение
    if count % 10 == 0:
        df_save = pd.concat([df_save, pd.DataFrame(cat_id_pred, columns=['hash_id', 'name', 'category_id'])])
        df_save.to_csv('unLabel_Qwen2.5-7B.csv', index=False, na_rep='NaN')
        cat_id_pred = []
    if count % 1000 == 0:
        print(df_save.shape)

# Сохраняем оставшиеся данные
if cat_id_pred:
    df_save = pd.concat([df_save, pd.DataFrame(cat_id_pred, columns=['category_id'])])
    df_save.to_csv('unLabel_Qwen2.5-7B.csv', index=False, na_rep='NaN')